***Author: Ritwika VPS, ritwika@ucmerced.edu***  
***Written: Dec 27, 2025***  

This file contains code to run robustness checks to show that for cases where task size $\geq$ total pooled memory at the group-level (i.e., T $\geq$ N_a m_a such that the task is, at best, minimally covered--for T = N_a m_a), the NoRedundRandom, NoRedund1Come1Serve, and NoRedundRandSeq task bit allocation protocols are functionally equivalent (and therefore, we can use the NoRedundRandom protocol for these cases across the board, from a computational implementation sense, as we have chosen to do in the core simulation code: a_M1_CIvsCM_MemortTaskSim.ipynb and the associated function file: Fns_MemoryTaskSim.py. Note that the RedundNoRepBits protocol is only different--operationally and functionally--from the NoRedundRandom protocol when T < N_a m_a and as such, the NoRedundRandom protocol is used to obtain results for the RedundNoRepBits protocol when T $\geq$ N_a m_a). This equivalence is because CM vs CI performance is not concerned with the identity of the task bits or the agent memory bits, and only with the degree of task coverage achieved. 

Also note that when T $\geq$ N_a m_a, the NoRedundRandom and NoRedund1Come1Serve protocols are operationally and functionally equivalent (again in terms of metrics we care about), and therefore, this equivalence demo does not test for equivalence between these to protocols.

(Admittedly, this demo comes out of choices made early on in how the core simulation code is set up: in the core sim code, for all T $\geq$ N_a m_a--i.e., as long as the pooled group memory is not greater than the task size, such that all agent memory bits are always assigned a task bit--task bit allocation is done using the NoRedundRandom protocol. This is based on the reasoning that operationally and functionally, i.e., for effects we care about--only how much of the task is covered in CI vs CM, not the details of how the task is covered, which bits are excluded, etc.--the NoRedundRandom protocol is equivalent to the NoRedund1Come1Serve, and NoRedundRandSeq protocols. However, for completeness's sake, it is good to verify this computationally, which is what this script does. Since the RedundNoRepBits protocol only kicks in when there are agent memory bits that can be redundantly assiged after full task coverage--and therefore, necessarily only when T < N_a m_a, there is no need to test this protocol against the others when T $\geq$ N_a m_a. At any rate, the core sims could have been set up differently by explicitly using NoRedundRandom, NoRedund1Come1Serve, and NoRedundRandSeq protocols when T $\geq$ N_a m_a as well as T < N_a m_a, but I am choosing not to change the core sims and instead, just doing these checks in this script)

In [49]:
"""
Import all necessary modules (importing modules for the function file as well, just to be safe)
"""
import Fns_MemoryTaskSim as CIvsCM
import numpy as np
import pandas as pd #data manipulation and analysis library
import os
from tqdm import tqdm #for progress bar
import scipy.io #to save and load data in .mat format (cuz I am doing the plotting in MATLAB)
import re #for regular expressions

In [50]:
def DistributeTask_LeqGroupMem_DistCondEquivTest(TaskSize, N_a, m_a, MemoryDistCondition):
    """
    This function distributes the task (consisting of TaskSize number of bits) among a group of N_a agents, each with memory size m_a bits such that N_a*m_a <= TaskSize.
    That is, the total number of memory bits pooled across agents in a group can at most only fully cover the TaskSize such that there are no redundant bits of memory.
    The task bit allocation protocol to agents is done either using the NoRedundRandom or NoRedundRandSeq protocols. We note that for N_a*m_a <= TaskSize, the NoRedund1Come1Serve
    protocol is operationally equivalent to the NoRedundRandom protocol, so we do not include the NoRedund1Come1Serve protocol here. The goal here is to simulate and verify that 
    both protocols are functionally equivalent re: the mean number of covered--and by extension, excluded--bits for the collective intellgience condition (the number of covered
    bits is fixed for the collective memopry condition)--ergo the 'EquivTest' suffix to the function.

    Inputs:
    TaskSize (int): memory size of task.
    N_a (int): Number of agents in a group.
    m_a (int): Memory capacity of each agent.
    MemoryDistCondition (string): Memory distribution condition; options are NoRedundRandom, NoRedundRandSeq

    Returns:
    AgentBits (2d numpy array): each row corresponds to an agent and the elements in the row correspond to the memory bits assigned to that agent.
    NumExcludedBits (int): Number of bits of the task not covered by the agents in the group.
    """

    TotalCoverage = N_a*m_a #Pooled memory

    #CHECK
    if TotalCoverage > TaskSize:
        raise ValueError("Pooled task memory is greater than task size.")

    MemoryPerm = np.random.permutation(range(TaskSize)) + 1 # Randomly permute the memory vector (obtained as range(TaskSize)) (+ 1 because Python indexes from 0)
    MemoryPermTrimmed = MemoryPerm[0:TotalCoverage] #Trim the permuted memory vector to only include the first TotalCoverage number of bits
    #(note that this will actually index from 0 to TotalCoverage-1 because Python indexes from 0)

    if MemoryDistCondition == "NoRedundRandom":
        AgentBits = np.reshape(MemoryPermTrimmed, (N_a, m_a))  #THIS IS EXACTLY OPERATIONALLY EQUIVALENT TO THE NoRedund1Come1Serve CASE when TotalCoverage <= TaskSize
    elif MemoryDistCondition == "NoRedundRandSeq":
        AgentBits = np.reshape(MemoryPermTrimmed, (m_a, N_a)).T #Reshape and then transpose to get the random sequential allocation
    else:
        raise ValueError("Invalid MemoryDistCondition. Possible values are: NoRedundRandom, NoRedundRandSeq.")
            
    NumExcludedBits = np.size(MemoryPerm) - np.size(AgentBits) #Number of bits excluded (should be TaskSize - TotalCoverage in this case)
    
    #error checks
    if NumExcludedBits != (TaskSize - TotalCoverage):
        raise ValueError("Number of bits excluded should be equal to TaskSize - TotalCoverage when TotalCoverage is less than TaskSize.")
    elif NumExcludedBits < 0:
        raise ValueError("Number of bits excluded should not be negative.")
    
    return AgentBits, NumExcludedBits

In [51]:
"""
This function computes the details of number of excluded bits for both CI and CM conditions based on the specified task bit-to-agent memory bit allocation protocol across N_g groups, 
for a given task size, agent number per group, and agent memory size, specficially for cases where TotalCoverage <= TaskSize. The goal is to demonstrate that the NoRedundRandom and 
NoRedundRandSeq allocation protocols are operationally and functionally equivalent (wrt to the metrics we care about; i.e., the number of excuded or covered task bits) when 
TotalCoverage <= TaskSize.

(This is adapated from similar functions in the Fns_MemoryTaskSim.py file, by specifically considering cases where TotalCoverage <= TaskSize ONLY)

Inputs:
TaskSize (int): memory size of task.
N_a (int): Number of agents in a group.
m_a (int): Memory capacity of each agent.
N_groups (int): Number of groups.
MemoryDistCondition (str): Condition for task memory distribution. 
    Possible values are: "NoRedundRandom", "NoRedundRandSeq"

Returns:
MeanExcludedBits, StdExcludedBits: mean and std dev of excluded bits across N_g shuffled groups (CI condition). 
TrueExcludedBits: number of excluded bits for stable groups (CM condition). Note that this is constant across all groups in the CM condition.
"""
def GetExcBitsCIandCM_LeqGroupMem_DistCondEquivTest(TaskSize, N_a, m_a, N_groups, MemoryDistCondition):
    
    TotalCoverage = N_a*m_a # Total memory coverage across agents
    TrueMemoryVec = np.arange(TaskSize) + 1 #The true memory vector (i.e., the range(TaskSize) vector) (+ 1 because Python indexes from 0)

    """
    Get task bit assignment for all N_g stable groups (CM condition) and the number of excluded bits (constant across all groups) for teh CM condition
    """
    AggregAgentBits = np.empty((0, m_a))  # Initialize an empty NumPy array with m_a columns (since each agent has m_a memory bits). We will use this to sequentially store
    #the AgentBits arrays obtained from each group, and then use this aggregated array to create shufflef groups
    ExcludedBitsArray_CM = np.zeros((N_groups,)) #Initialize arrays to store the number of excluded bits across N_groups number of runs for CM and CI conditions
    ExcludedBitsArray_CI = np.zeros((N_groups,)) 

    if TotalCoverage <= TaskSize: #Case when total task coverage is at most the same as TaskSize
        for i in range(N_groups): #Go through each group and test for different conditions
            AgentBits, ExcludedBitsArray_CM[i] = DistributeTask_LeqGroupMem_DistCondEquivTest(TaskSize, N_a, m_a, MemoryDistCondition)  
            AggregAgentBits = np.vstack([AggregAgentBits, AgentBits]) #Append the AgentBits array to the AggregAgentBits array
    else: #Case when total task coverage is greater than TaskSize
        raise ValueError("Total coverage is greater than task size.")

    if np.size(np.unique(ExcludedBitsArray_CM)) > 1:
        raise ValueError('Number of excluded bits varies across groups. This should not happen') #cuz the excluded bits should be the same across all groups in stable 
    #group formation scenarios
    
    TrueExcludedBits = np.unique(ExcludedBitsArray_CM)[0] #get the number of excluded bits for stable groups

    """
    Get shuffled groups (CI condition)
    """
    NumRows = np.shape(AggregAgentBits)[0] #Get the number of rows in the AggregAgentBits array (this will be the actual length, not indexed from 0. So, a value of 3 will mean there
    #are 3 rows, not 2)
    ShuffledGroupInds = np.random.permutation(range(NumRows)) #Get a random permutation of the row indices of the AggregAgentBits array
    #Note that because we are using range(NumRows), the indices are indexed from 0. So, if NumRows = 3, the possible indices are [0, 1, 2], which will then be randomly permuted
    ShuffledGroups = AggregAgentBits[ShuffledGroupInds] #Shuffle the groups by indexing the AggregAgentBits array with the randomly permuted row indices

    #Error check!
    if NumRows != np.shape(ShuffledGroups)[0]:
        raise ValueError("Number of rows in ShuffledGroups should be the same as that in AggregAgentBits.")
    
    """
    Get excluded bits for CI condition for each group to get mean and std dev
    """
    for i in range(N_groups):
        CurrShuffledGroup = ShuffledGroups[i*N_a:(i+1)*N_a, :] #Get the current shuffled group by indexing the ShuffledGroups array.
        #Note that when python indexes from i to j, what we actually get is i to (j-1), which is why the above indexing works (which is wild and a lil confusing to me)
        NumExcludedBits_ShuffledGroup = np.size(np.setdiff1d(TrueMemoryVec, CurrShuffledGroup)) #Get the number of excluded bits for the current shuffled group
        #Note that as set up here, is agnostic to NaNs in AgentBits, because we are extracting values that are in TrueMemoryVec but not in AgentBits
        ExcludedBitsArray_CI[i] = NumExcludedBits_ShuffledGroup 

    MeanExcludedBits = np.mean(ExcludedBitsArray_CI) #Mean of excluded bits across N_groups number of runs (setdiff already takes care of NaNs; CI condition)
    StdExcludedBits = np.std(ExcludedBitsArray_CI) #Standard deviation of excluded bits across N_groups number of runs (CI condition)
        
    return MeanExcludedBits, StdExcludedBits, TrueExcludedBits

In [52]:
""" 
This function gets CI vs CM stats for a parameter sweep across different N_a and m_a values, specifically for cases where TotalCoverage <= TaskSize. We do not parallelise this function
because by limiting exploration to cases when TotalCoverage <= TaskSize, the computational cost is severaly diminished. The goal here is to demonstrate the functional equivalence of 
the NoRedundRandom and the NoRedundRandSeq allocation protocols re: the number of excluded bits across different parameter combinations (in terms of metrics we care about: number of 
excluded or covered task bits under CI and CM conditions) SPECIFICALLY for cases where TotalCoverage <= TaskSize.

Inputs:
TaskSize (int): memory size of task.
N_a_vec (int): Vector of number of agents in a group.
m_a_vec (int): vector of memory capacity of each agent.
N_groups (int): Number of groups.
MemoryDistCondition (str): Condition for task memory distribution. 
    Possible values are: "NoRedundRandom", "NoRedundRandSeq"

Returns:
MeanExcludedBitsArray, StdExcludedBitsArray: 2d numpy arrays containing mean and std dev of excluded bits across N_g shuffled groups (CI condition), for all N_a and m_a values in the
                                             parameter sweep such that TaskSize >= N_a*m_a. 
TrueExcludedBits: 2d numpy array containing number of excluded bits for stable groups (CM condition) for all N_a and m_a values in the parameter sweep such that TaskSize >= N_a*m_a

Note that for all outputs, when TaskSize < N_a*m_a, the corresponding entries in the output arrays are NaN.
"""
def GetCIvsCMstatsForParamSweep_LeqGroupMem_DistCondEquivTest(TaskSize, N_a_vec, m_a_vec, N_g, MemoryDistCondition):

    #Initialise arrays
    NumAgents = len(N_a_vec)
    NumAgentMemBits = len(m_a_vec)
    MeanExcludedBitsArray = np.full((NumAgents, NumAgentMemBits), np.nan) #np.zeros((len(TaskMem_To_Na), len(TaskMem_To_ma)))
    StdExcludedBitsArray = np.full((NumAgents, NumAgentMemBits), np.nan)
    Stable_ExcludedBitsArray = np.full((NumAgents, NumAgentMemBits), np.nan)

    #print(type(MeanExcludedBitsArray), type(StdExcludedBitsArray), type(Stable_ExcludedBitsArray))

    for i_Na in tqdm(range(NumAgents), total=NumAgents):
        for i_ma in range(NumAgentMemBits):

            N_a = N_a_vec[i_Na] #Number of agents in a group
            m_a = m_a_vec[i_ma] #Memory size of each agent

            #print(N_a, m_a)
            if N_a*m_a <= TaskSize: #only proceed if task size is greater than total pooled memory at the group-level
                MeanTemp, StdTemp, StableExcBitsTemp = GetExcBitsCIandCM_LeqGroupMem_DistCondEquivTest(TaskSize, N_a, m_a, N_g, MemoryDistCondition)

                MeanExcludedBitsArray[i_Na,i_ma] = MeanTemp
                StdExcludedBitsArray[i_Na,i_ma] = StdTemp
                Stable_ExcludedBitsArray[i_Na,i_ma] = StableExcBitsTemp

    #print(type(MeanTemp), type(StdTemp), type(StableExcBitsTemp))
    return MeanExcludedBitsArray, StdExcludedBitsArray, Stable_ExcludedBitsArray 

In [53]:
""" 
Main equivalence test simulation
"""
#Load .mat file from core simulation results. It does not matter which allocation protocol we are loading results from, cuz this is just to get the agent memory vector, number of 
#agents vector, task size, and number of groups
#---------CHANGE PATH ACCORDINGLY------------------#
CoreSimResultsMat = scipy.io.loadmat('/Users/ritwikavps/Desktop/GoogleDriveFiles/research/CollectiveMemoryvsCollectiveIntelligence/MemoryGameData/CI_vs_CM_Sims_MemoryDistCondition__NoRedund1Come1Serve.mat')
#--------------------------------------------------#

#Get variables/numbers to pass to simulation
TaskSize = CoreSimResultsMat['TaskSize'][0][0]
N_g = CoreSimResultsMat['NumGroups'][0][0]
N_a_vec = CoreSimResultsMat['NumAgents_Vec'][0]
m_a_vec = CoreSimResultsMat['AgentMemory_Vec'][0]

MemoryDistCond_Vec = ['NoRedundRandom', 'NoRedundRandSeq'] 

for MemoryDistCond in MemoryDistCond_Vec: #do equivalence test sim for the memory distributioin conditions
    
    print(f'Starting sim for {MemoryDistCond}')

    MeanExcludedBitsArray, StdExcludedBitsArray, Stable_ExcludedBitsArray = GetCIvsCMstatsForParamSweep_LeqGroupMem_DistCondEquivTest(TaskSize, N_a_vec, m_a_vec, N_g, MemoryDistCond)
    
    DataToSave = { #organise the data to be saved
    'MeanExcBits_CI': MeanExcludedBitsArray,
    'StdExcBits_CI': StdExcludedBitsArray,
    'StableExcBits_CM': Stable_ExcludedBitsArray,
    'NumAgents_Vec': N_a_vec,
    'AgentMemory_Vec': m_a_vec,
    'TaskSize': TaskSize,
    'MemoryDistributionCondition': MemoryDistCond,
    'NumGroups': N_g
    }
    FileName = 'CI_vs_CM_Sims_MemoryDistCondEquivTest_LeqGroupMem__' + MemoryDistCond +'.mat' # note that the data files will be saved in the current working directory
    scipy.io.savemat(FileName, DataToSave) #save .mat file




Starting sim for NoRedundRandom


100%|██████████| 22/22 [00:10<00:00,  2.08it/s]


Starting sim for NoRedundRandSeq


100%|██████████| 22/22 [00:10<00:00,  2.13it/s]
